In [1]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


## 环境变量

In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# print(os.environ)

AI_TEMPERATURE=0.0
AI_MAX_TOKENS=2048

## 模型加载

In [3]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI

def load_azureLLM():
    # chat
    chat = AzureChatOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_CHAT'),
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    # llm
    llm = AzureOpenAI(
        deployment_name=os.environ.get('DEPLOYMENT_NAME_LLM'),
        model_name="text-davinci-003",
        temperature=AI_TEMPERATURE,
        max_tokens=AI_MAX_TOKENS,
        )
    
    embedding = OpenAIEmbeddings(deployment = os.environ.get('DEPLOYMENT_NAME_EMBEDDING'),chunk_size=1)
    
    return llm,chat,embedding

llm,chat,embed = load_azureLLM()

In [4]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.agents import initialize_agent

from langchain.memory import ConversationBufferMemory

In [5]:

memory = ConversationBufferMemory(memory_key="chat_history")

In [6]:
from langchain.chains import LLMMathChain

llm_math_chain = LLMMathChain.from_llm(llm=llm,verbose=True)
tools = [
    Tool(
        name = "计算器",
        func=llm_math_chain.run,
        description="当你需要数学计算时，这个工具很有用"
    ),
]

In [7]:
agent_chain = initialize_agent(
    tools=tools,
    llm=llm,
    memory=memory,
    agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    verbose=True)

In [8]:
agent_chain.run(input="hello, i'm linda!")

Error in on_chain_start callback: 'name'



Thought: Do I need to use a tool? No
AI: Hi Linda, nice to meet you! How can I help you today?

> Finished chain.


'Hi Linda, nice to meet you! How can I help you today?'

In [9]:
agent_chain.run(input="2+5等于多少？")

Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'



Thought: Do I need to use a tool? Yes
Action: 计算器
Action Input: 2+52+5```text
2+5
```
...numexpr.evaluate("2+5")...

Answer: 7
> Finished chain.

Observation: Answer: 7
Thought: Do I need to use a tool? No
AI: The answer is 7.

> Finished chain.


'The answer is 7.'

In [10]:
agent_chain.run(input="我叫什么？")

Error in on_chain_start callback: 'name'


Thought: Do I need to use a tool? No
AI: You said your name is Linda. Is that correct?

> Finished chain.


'You said your name is Linda. Is that correct?'

In [11]:
agent_chain.run(input="今天会下雨吗？")

Error in on_chain_start callback: 'name'


Thought: Do I need to use a tool? No
AI: Unfortunately, I'm not able to answer that question. However, you can check the weather forecast for your area to get an accurate prediction.

> Finished chain.


"Unfortunately, I'm not able to answer that question. However, you can check the weather forecast for your area to get an accurate prediction."

In [12]:
agent_chain.run(input="2的平方根是多少？")

Error in on_chain_start callback: 'name'
Error in on_chain_start callback: 'name'



Thought: Do I need to use a tool? Yes
Action: 计算器
Action Input: 2的平方根2的平方根```text
sqrt(2)
```
...numexpr.evaluate("sqrt(2)")...

Answer: 1.4142135623730951
> Finished chain.

Observation: Answer: 1.4142135623730951
Thought: Do I need to use a tool? No
AI: The square root of 2 is 1.4142135623730951.

> Finished chain.


'The square root of 2 is 1.4142135623730951.'

In [13]:
agent_chain.run(input="我叫什么？")

Error in on_chain_start callback: 'name'



Thought: Do I need to use a tool? No
AI: You said your name is Linda. Is that correct?

> Finished chain.


'You said your name is Linda. Is that correct?'

In [14]:
print(memory.buffer)

Human: hello, i'm linda!
AI: Hi Linda, nice to meet you! How can I help you today?
Human: 2+5等于多少？
AI: The answer is 7.
Human: 我叫什么？
AI: You said your name is Linda. Is that correct?
Human: 今天会下雨吗？
AI: Unfortunately, I'm not able to answer that question. However, you can check the weather forecast for your area to get an accurate prediction.
Human: 2的平方根是多少？
AI: The square root of 2 is 1.4142135623730951.
Human: 我叫什么？
AI: You said your name is Linda. Is that correct?
